In [ ]:
# default_exp int_test_helpers

# YX_Motor Integration Tests

Only run these when you are able to connect to an actual Alteryx Analytics Hub test instance.  

The creds for this instance are expected to be stored in a file called "private.py" stored in the local development directory.

## Setup and Client Tests

In [ ]:
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

######################### Integration Test Scaffolding ############################
from yx_motor.client import Client

from private import server_vars

base_url=server_vars['base_url']
login_email = server_vars['login_email']
login_pwd = server_vars['login_pwd']

dev_client = Client(base_url=base_url,
                    login_email=login_email,
                    login_pwd=login_pwd)

assert(dev_client.api.is_authenticated)

POST sent to: https://hub-beta.demo.alteryx.com/api/v1/authenticate/
with headers: {'User-Agent': 'python-requests/2.23.0', 'Accept-Encoding': 'gzip,deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Type': 'application/json', 'Accept-Language': 'en-US,en;q=0.5', 'Content-Length': '57'}
Response Status: 200


## Jobs Integration Tests

In [ ]:
job_id = '337ce711-bdcf-4f33-a2f6-81c36a727c58'
dev_client.jobs.get_log(job_id=job_id).json()

GET sent to: https://test.aep-mono.devops.alteryx.com/api/v1/jobs/337ce711-bdcf-4f33-a2f6-81c36a727c58/logs
with headers: {'User-Agent': 'python-requests/2.23.0', 'Accept-Encoding': 'gzip,deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Type': 'application/json', 'Accept-Language': 'en-US,en;q=0.5', 'Cookie': 'ayxSession=s%3A704beafc-29ca-4668-a8fd-cbd7841ee1d4.40ZjMFl93ECXor7wdXaDK%2BJrdFpvL8ZdW8qhIq6ccfQ'}
Response Status: 404


{'requestUuid': '4a94be12-4e46-406b-9ecd-16ae55d7f686',
 'errors': [{'code': '',
   'message': 'JobOrchestrator.getJobLogs() with args: [ { "requestUuid": "4a94be12-4e46-406b-9ecd-16ae55d7f686","auditEvents": "[  ]","privateSession": "{ NESTED OBJECT }","privateSiteId": "b976ae02-6539-45b8-8341-0b0dd57d3b42","clientIp": "::ffff:192.168.62.74" },"337ce711-bdcf-4f33-a2f6-81c36a727c58", { "schema": "{ NESTED OBJECT }","schemaErrorCode": "{ NESTED OBJECT }","from": 0,"pageSize": 10,"sortBy": "[  ]","filterStrategy": "AND" }]',
   'debugInfo': 'PlatformError: JobOrchestrator.getJobLogs() with args: [ { "requestUuid": "4a94be12-4e46-406b-9ecd-16ae55d7f686","auditEvents": "[  ]","privateSession": "{ NESTED OBJECT }","privateSiteId": "b976ae02-6539-45b8-8341-0b0dd57d3b42","clientIp": "::ffff:192.168.62.74" },"337ce711-bdcf-4f33-a2f6-81c36a727c58", { "schema": "{ NESTED OBJECT }","schemaErrorCode": "{ NESTED OBJECT }","from": 0,"pageSize": 10,"sortBy": "[  ]","filterStrategy": "AND" }]\n    at 

In [ ]:
from yx_motor import int_test_helpers as helpers

def run_jobs_integration_tests(client):
    helpers.test_get_log_functionality(client)
    helpers.test_can_get_jobs_with_no_args(client)
    helpers.test_that_server_has_jobs_available(client)
    helpers.test_can_get_job_by_id(client)
    helpers.test_cancel_job_functionality(client)

run_jobs_integration_tests(dev_client)

GET sent to: https://hub-beta.demo.alteryx.com/api/v1/jobs/
with headers: {'User-Agent': 'python-requests/2.23.0', 'Accept-Encoding': 'gzip,deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Type': 'application/json', 'Accept-Language': 'en-US,en;q=0.5', 'Cookie': 'ayxSession=s%3Acf360ce3-7fa1-46a5-a14f-ba1411959855.gY6I7leq%2B1QRCAuE2PT1ww2tf3oDpr6cNDISsUwqn54'}
Response Status: 200
GET sent to: https://hub-beta.demo.alteryx.com/api/v1/jobs/9b724581-6796-4c91-9400-5cd7ed21ed47/logs
with headers: {'User-Agent': 'python-requests/2.23.0', 'Accept-Encoding': 'gzip,deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Type': 'application/json', 'Accept-Language': 'en-US,en;q=0.5', 'Cookie': 'ayxSession=s%3Acf360ce3-7fa1-46a5-a14f-ba1411959855.gY6I7leq%2B1QRCAuE2PT1ww2tf3oDpr6cNDISsUwqn54'}
Response Status: 200
GET sent to: https://hub-beta.demo.alteryx.com/api/v1/jobs/
with headers: {'User-Agent': 'python-requests/2.23.0', 'Accept-Encoding': 'gzip,deflate', 'Accept':

## Authenticate Integration Tests

In [ ]:
def scaffold_api_with_creds():
    pass

def test_can_log_in():
    pass

def test_can_log_out():
    pass

## Files Integration Tests

## Workflows Integration Tests

In [ ]:
tlist = [1, 2, 3, 4]

In [ ]:
all([
    x > 0 for x in tlist
])

In [ ]:
from nbdev.export import *
notebook2script()

Converted 01_client.ipynb.
Converted 02_jobs.ipynb.
Converted 03_api.ipynb.
Converted 04_authenticate.ipynb.
Converted 05_files.ipynb.
Converted 06_workflows.ipynb.
Converted index.ipynb.
Converted integration_tests.ipynb.
